In [13]:
import xml.etree.ElementTree as ET
import pandas as pd

Defining rules

In [14]:
log_df = pd.read_csv('log.csv', sep=';', usecols=['EventName','ID'])

# Function to parse the DCR XML and extract rules
def parse_dcr_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
log_df = log_df.groupby('ID')['EventName'].apply(list)

print(log_df)

ID
14a-026_0    [Fill out application, Approved - to board, Ch...
14a-027_0    [Fill out application, Approved - to board, Ch...
14a-031_2    [Fill out application, Screening reject, Appli...
14a-035_0    [Fill out application, Approved - to board, Ch...
14a-037_0    [Fill out application, Approved - to board, Ch...
                                   ...                        
14b-627_0    [Fill out application, Approved - to board, Ch...
14b-627_1    [Fill out application, Approved - to board, Ch...
14b-628_0    [Fill out application, Approved - to board, Ch...
14b-629_1    [Fill out application, Approved - to board, Ch...
14b-631_0    [Fill out application, Approved - to board, Ch...
Name: EventName, Length: 594, dtype: object


In [15]:
def check_conformance(log_df):
    
    
    rule_counts = {
        "Rule 1": {"Satisfied": 0, "Not Satisfied": 0},
        "Rule 2": {"Satisfied": 0, "Not Satisfied": 0},
        "Rule 3": {"Satisfied": 0, "Not Satisfied": 0},
        "Rule 4": {"Satisfied": 0, "Not Satisfied": 0},
        "Rule 5": {"Satisfied": 0, "Not Satisfied": 0},
        "Rule 6": {"Satisfied": 0, "Not Satisfied": 0},
        "Rule 7": {"Satisfied": 0, "Not Satisfied": 0},
        "Rule 8": {"Satisfied": 0, "Not Satisfied": 0},
    }

    
    

# Iterate through each event in the log and check rules    
    for events in log_df:
            for i, event in enumerate(events):
                # Rule 1: 'Fill out application' should always be the first event
                if i == 0:
                    if event == "Fill out application":
                        rule_counts["Rule 1"]["Satisfied"] +=1
                    else:
                        rule_counts["Rule 1"]["Not Satisfied"] += 1

                     
            # Rule 2: 'Lawyer Review' and 'Architect Review' should never occur together
            if "Lawyer Review" in events and "Architect Review" in events:
                rule_counts["Rule 2"]["Not Satisfied"] += 1
            else:
                rule_counts["Rule 2"]["Satisfied"] += 1

            # Rule 3: Reject should always eventually be followed by Applicant informed and Change phase to Abort. 
            if "Reject" in events:
                reject_index = events.index("Reject")
                # Check if either "Applicant informed" or "Change phase to Abort" occurs after "Reject"
                if "Applicant informed" not in events[reject_index] and "Change phase to Abort" not in events[reject_index]:
                    rule_counts["Rule 3"]["Not Satisfied"] += 1
            else:
                rule_counts["Rule 3"]["Satisfied"] += 1


            # Rule 4: First payment should only occur once, unless Undo payment is executed afterwards, in which case it may be repeated once.
            first_payment_count = events.count("First payment")
            if first_payment_count == 1 or (first_payment_count == 2 and "Undo payment" in events):
                rule_counts["Rule 4"]["Satisfied"] += 1
            elif first_payment_count == 0:
                rule_counts["Rule 4"]["Satisfied"] += 1  # Satisfies the rule by default
            elif first_payment_count not in [0, 1, 2]:
                rule_counts["Rule 4"]["Not Satisfied"] += 1
            else:
                rule_counts["Rule 4"]["Not Satisfied"] += 1
            
            # Rule 5: If Account number changed happens, then afterwards Approve changed account needs to 
            # be executed before one can execute First payment
            if "Account number changed" in events:
                account_index = events.index("Account number changed")
                if "Approve changed account" not in events[account_index:] or "First payment" not in events[account_index:]:
                    rule_counts["Rule 5"]["Not Satisfied"] += 1
            else:
                rule_counts["Rule 5"]["Satisfied"] += 1

            # Rule 6: Change Phase to Payout should always be followed by First payment
            if "Change Phase to Payout" in events:
                payout_index = events.index("Change Phase to Payout")
                if "First payment" not in events[payout_index:]:
                    rule_counts["Rule 6"]["Not Satisfied"] += 1
                else:
                    rule_counts["Rule 6"]["Satisfied"] += 1
            else:
                rule_counts["Rule 6"]["Satisfied"] += 1

            # Rule 7: Restrictions for Change Phase to End Report after Payout and First payment
            if "Change Phase to Payout" in events:
                payout_index = events.index("Change Phase to Payout")
                if "First payment" in events[payout_index:]:
                    if "Change Phase to End Report" in events[payout_index:]:
                        report_index = events.index("Change Phase to End Report")
                        if "First payment" not in events[report_index:]:
                            rule_counts["Rule 7"]["Not Satisfied"] += 1
                        else:
                            print("How did I get here?")
                    else:
                        rule_counts["Rule 7"]["Not Satisfied"] += 1
                elif "Change Phase to End Report" in events[payout_index:]: 
                    rule_counts["Rule 7"]["Not Satisfied"] += 1
                else:
                    rule_counts["Rule 7"]["Satisfied"] += 1
            else:
                rule_counts["Rule 7"]["Satisfied"] += 1

            # Rule 8: Execute Abandon and Change phase to Abandon
            if "Execute abandon" in events:
                abandon_index = events.index("Execute abandon")
                if "Change phase to abandon" not in events[abandon_index:]:
                    rule_counts["Rule 8"]["Not Satisfied"] += 1
            else:
                rule_counts["Rule 8"]["Satisfied"] += 1

    # Print or return the counts
    #print("This is what events mean:",events)
    #print("This is what event means:",event)
    return rule_counts

# conformance check
result = check_conformance(log_df)
print("Conformance Check Results:")
for rule, counts in result.items():
    total = counts['Satisfied'] + counts['Not Satisfied']
    print(f"{rule}: Satisfied {counts['Satisfied']}, Not Satisfied {counts['Not Satisfied']}, Total {total}")

Conformance Check Results:
Rule 1: Satisfied 594, Not Satisfied 0, Total 594
Rule 2: Satisfied 305, Not Satisfied 289, Total 594
Rule 3: Satisfied 369, Not Satisfied 225, Total 594
Rule 4: Satisfied 594, Not Satisfied 0, Total 594
Rule 5: Satisfied 591, Not Satisfied 3, Total 594
Rule 6: Satisfied 592, Not Satisfied 2, Total 594
Rule 7: Satisfied 576, Not Satisfied 18, Total 594
Rule 8: Satisfied 592, Not Satisfied 2, Total 594


Parsing and defining xml file

In [16]:
def parse_xml_file(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    print("Root tag:", root.tag)
    for child in root.findall("labelMapping"):
        print("Child tag:", child.tag)

    mapping = dict()
    for activity in root.find("specification").find("resources").find("labelMappings").findall("labelMapping"):
        #print(activity.get("labelId"))
        #print(activity.get("eventId"))
        mapping[activity.get("labelId")] =activity.get("eventId") 
    print(mapping)
xml_file_path = "DCR-Task-2.xml"

parse_xml_file(xml_file_path)

Root tag: dcrgraph
{'Fill out application': 'Activity0', 'Lawyer review': 'Activity1', 'Architect Review': 'Activity2', 'Change phase to abandon': 'Activity3', 'Execute abandon': 'Activity4', 'Approve': 'Activity5', 'Reject': 'Activity6', 'Approve changed account': 'Activity7', 'Account number changed': 'Activity8', 'Change phase to payout': 'Activity9', 'First payment': 'Activity10', 'Undo Payment': 'Activity11', 'Change phase to aborted': 'Activity12', 'Applicant informed': 'Activity13', 'Change phase to end report': 'Activity14', 'Receive end report': 'Activity15', 'Change phase to completed': 'Activity16'}


Conformance check 

In [17]:

#print(log_df)
#parse_xml_file(xml_file_path)

pattern_counts = {
        "Pattern 1": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 2": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 3": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 4": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 5": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 6": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 7": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 8": {"Satisfied": 0, "Not Satisfied": 0,},
    }


def parse_respons(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()



    mapping2 = dict()
    for activity in root.find("specification").find("constraints").find("responses").findall("response"):
    #print(activity.get("labelId"))
    #print(activity.get("eventId"))
        mapping2[activity.get("sourceId")] = activity.get("targetId") 
    print(mapping2)

parse_respons(xml_file_path)


def pattern1_response(event, rules):
    if event in rules['responses']:
        return rule
    return None



{'Activity0': 'Activity2', 'Activity1': 'Activity6', 'Activity2': 'Activity6', 'Activity6': 'Activity12', 'Activity5': 'Activity8', 'Activity4': 'Activity3', 'Activity15': 'Activity16', 'Activity8': 'Activity7', 'Activity7': 'Activity9', 'Activity9': 'Activity10', 'Activity10': 'Activity14', 'Activity14': 'Activity15'}
